In [5]:
pip install mtcnn

Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
import cv2
import numpy as np
from mtcnn import MTCNN
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import img_to_array

# Initialize MTCNN face detector
detector = MTCNN()

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# load the model
model.load_weights('trained_model.h5')
def extract_faces_from_video(video_path, target_size=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    faces = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        detected_faces = detector.detect_faces(frame_rgb)

        for face in detected_faces:
            x, y, width, height = face['box']
            face_img = frame_rgb[y:y+height, x:x+width]
            face_img = cv2.resize(face_img, target_size)
            face_array = img_to_array(face_img) / 255.0
            faces.append(face_array)
    
    cap.release()
    return np.array(faces)

# Function to predict if the video is fake or real
def predict_video(video_path):
    faces = extract_faces_from_video(video_path)
    if len(faces) == 0:
        print("No faces detected in the video.")
        return None

    # Reshape faces to match the input shape expected by the model
    faces = np.reshape(faces, (faces.shape[0], faces.shape[1], faces.shape[2], faces.shape[3]))

    predictions = model.predict(faces)
    avg_prediction = np.mean(predictions)

    if avg_prediction > 0.5:
        print(f"The video '{video_path}' is predicted to be FAKE.")
    else:
        print(f"The video '{video_path}' is predicted to be REAL.")

    return avg_prediction

# Test the prediction function with a sample video
video_path = r'C:\Users\HP\Downloads\Deepfake\fake-agotmizucf-0.mp4'  # Update with the actual path to the test video
predict_video(video_path)


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 169ms/step
The video 'C:\Users\HP\Downloads\Deepfake\fake-agotmizucf-0.mp4' is predicted to be FAKE.


0.99980915